# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 2 2023 2:55PM,258584,19,ACG-2102492970,ACG North America LLC,Released
1,Mar 2 2023 2:47PM,258583,10,MSP219109,"Methapharm, Inc.",Released
2,Mar 2 2023 2:47PM,258583,10,MSP219110,"Methapharm, Inc.",Released
3,Mar 2 2023 2:39PM,258580,10,9225321,"Akron BioProducts, LLC",Released
4,Mar 2 2023 2:39PM,258582,10,PRF-54587,Bio-PRF,Released
5,Mar 2 2023 2:39PM,258582,10,PRF-54590,Bio-PRF,Released
6,Mar 2 2023 2:39PM,258582,10,PRF-54593,Bio-PRF,Released
7,Mar 2 2023 2:39PM,258582,10,PRF-54596,Bio-PRF,Released
8,Mar 2 2023 2:39PM,258582,10,PRF-54599,Bio-PRF,Released
9,Mar 2 2023 2:39PM,258582,10,PRF-54602,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,258579,Released,1
24,258580,Released,1
25,258582,Released,6
26,258583,Released,2
27,258584,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
258579,NaN,NaN,1.0
258580,NaN,NaN,1.0
258582,NaN,NaN,6.0
258583,NaN,NaN,2.0
258584,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258461,15.0,13.0,5.0
258487,0.0,1.0,0.0
258495,0.0,1.0,2.0
258498,0.0,0.0,1.0
258523,0.0,0.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
258461,15,13,5
258487,0,1,0
258495,0,1,2
258498,0,0,1
258523,0,0,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258461,15,13,5
1,258487,0,1,0
2,258495,0,1,2
3,258498,0,0,1
4,258523,0,0,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,258461,15,13,5
1,258487,,1,
2,258495,,1,2
3,258498,,,1
4,258523,,,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc."
3,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC"
4,Mar 2 2023 2:39PM,258582,10,Bio-PRF
10,Mar 2 2023 2:34PM,258579,10,Emerginnova
11,Mar 2 2023 2:33PM,258578,10,"Senseonics, Incorporated"
12,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated"
13,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc."
14,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix
15,Mar 2 2023 1:46PM,258568,16,Capsugel


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC,,,1
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc.",,,2
2,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC",,,1
3,Mar 2 2023 2:39PM,258582,10,Bio-PRF,,,6
4,Mar 2 2023 2:34PM,258579,10,Emerginnova,,,1
5,Mar 2 2023 2:33PM,258578,10,"Senseonics, Incorporated",,,1
6,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",,,1
7,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc.",,,1
8,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix,,,1
9,Mar 2 2023 1:46PM,258568,16,Capsugel,,1,


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC,1,,
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc.",2,,
2,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC",1,,
3,Mar 2 2023 2:39PM,258582,10,Bio-PRF,6,,
4,Mar 2 2023 2:34PM,258579,10,Emerginnova,1,,
5,Mar 2 2023 2:33PM,258578,10,"Senseonics, Incorporated",1,,
6,Mar 2 2023 1:58PM,258563,10,"Senseonics, Incorporated",1,,
7,Mar 2 2023 1:48PM,258572,19,"Granules Pharmaceuticals, Inc.",1,,
8,Mar 2 2023 1:46PM,258569,10,Yusen – 3D Matrix,1,,
9,Mar 2 2023 1:46PM,258568,16,Capsugel,,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC,1,,
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc.",2,,
2,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC",1,,
3,Mar 2 2023 2:39PM,258582,10,Bio-PRF,6,,
4,Mar 2 2023 2:34PM,258579,10,Emerginnova,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC,1.0,NaN,NaN
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc.",2.0,NaN,NaN
2,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC",1.0,NaN,NaN
3,Mar 2 2023 2:39PM,258582,10,Bio-PRF,6.0,NaN,NaN
4,Mar 2 2023 2:34PM,258579,10,Emerginnova,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Mar 2 2023 2:55PM,258584,19,ACG North America LLC,1.0,0.0,0.0
1,Mar 2 2023 2:47PM,258583,10,"Methapharm, Inc.",2.0,0.0,0.0
2,Mar 2 2023 2:39PM,258580,10,"Akron BioProducts, LLC",1.0,0.0,0.0
3,Mar 2 2023 2:39PM,258582,10,Bio-PRF,6.0,0.0,0.0
4,Mar 2 2023 2:34PM,258579,10,Emerginnova,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2585719,18.0,0.0,0.0
12,258487,0.0,1.0,0.0
15,1034047,21.0,14.0,15.0
16,517103,1.0,1.0,0.0
19,1809796,26.0,0.0,0.0
22,258498,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2585719,18.0,0.0,0.0
1,12,258487,0.0,1.0,0.0
2,15,1034047,21.0,14.0,15.0
3,16,517103,1.0,1.0,0.0
4,19,1809796,26.0,0.0,0.0
5,22,258498,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,18.0,0.0,0.0
1,12,0.0,1.0,0.0
2,15,21.0,14.0,15.0
3,16,1.0,1.0,0.0
4,19,26.0,0.0,0.0
5,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,18.0
1,12,Released,0.0
2,15,Released,21.0
3,16,Released,1.0
4,19,Released,26.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Completed,0.0,0.0,15.0,0.0,0.0,0.0
Executing,0.0,1.0,14.0,1.0,0.0,0.0
Released,18.0,0.0,21.0,1.0,26.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Completed,0.0,0.0,15.0,0.0,0.0,0.0
1,Executing,0.0,1.0,14.0,1.0,0.0,0.0
2,Released,18.0,0.0,21.0,1.0,26.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Completed,0.0,0.0,15.0,0.0,0.0,0.0
1,Executing,0.0,1.0,14.0,1.0,0.0,0.0
2,Released,18.0,0.0,21.0,1.0,26.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()